In [15]:
from langchain.agents import create_agent #for creating the agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit #for SQL database toolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate #for creating chat prompts templates
from langchain_google_genai import ChatGoogleGenerativeAI #for using Google Gemini LLM API
from dotenv import load_dotenv
import os

load_dotenv('../credentials.env') #load environment variables (credentials and API keys) from a .env file


True

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.getenv("GOOGLE_API_KEY")
    )

In [18]:
system_prompt = """You are a SQL expert assistant for an expense tracking app. Follow these instructions:
- Always verify table schemas before querying
- Use proper MySQL syntax
- Format results clearly and concisely
- If no data exists, state that explicitly
- Double-check date ranges in queries
- Provide short, actionable insights, tips, or budget suggestions when relevant
- Keep explanations brief and focused on practical guidance"""


In [25]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{messages}"),
])

In [26]:
chain = prompt | llm

In [ ]:
mysql_uri = f'mysql+mysqlconnector://{os.getenv("MYSQL_USERNAME")}:{os.getenv("MYSQL_PASSWORD")}@{os.getenv("MYSQL_HOST")}:{os.getenv("MYSQL_PORT")}/{os.getenv("DATABASE_NAME")}'
db = SQLDatabase.from_uri(mysql_uri) # type: ignore

toolkit = SQLDatabaseToolkit(db=db, llm=llm) # setting up the SQL toolkit
agent = create_agent(chain, toolkit.get_tools) # creating the agent

In [32]:
agent.invoke({
    "messages": [
        {"role": "user", "content": "Show me the total expenses for each category in the last month, if no data or even a database, just say that."}
    ]
})


{'messages': [HumanMessage(content='Show me the total expenses for each category in the last month, if no data or even a database, just say that.', additional_kwargs={}, response_metadata={}, id='5cf1db8a-d30e-4bfa-8dc7-2557761248f2'),
  AIMessage(content="Let's look at your spending by category for the last month.\n\n```sql\nSELECT\n  c.category_name,\n  SUM(e.amount) AS total_spent\nFROM expenses AS e\nJOIN categories AS c\n  ON e.category_id = c.category_id\nWHERE\n  e.expense_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)\nGROUP BY\n  c.category_name\nORDER BY\n  total_spent DESC;\n```\n\n**Insight:** Reviewing this report can help you identify where most of your money is going. Consider setting category-specific budget limits if you notice any categories that are consistently high.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'grounding_m